In [6]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

## XML File process
* Load all xml files and store them in a list
* Read xml files

In [32]:
xml_files = glob('datasets/Annotation/*/*.xml')

In [33]:
xml_files

['datasets/Annotation/n02097658-silky_terrier/n02097658_595.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_4869.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_26.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_422.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_9222.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_7970.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_1326.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_1469.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_10567.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_5223.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_1910.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_1086.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_2766.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_3693.xml',
 'datasets/Annotation/n02097658-silky_terrier/n02097658_10997.xml

In [43]:
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()
    
    # extract filename
    image_name = f"{root.find('filename').text}.jpg"
    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name, xmin, xmax, ymin, ymax])
        
    return parser

In [44]:
parser_all = list(map(extract_text, xml_files))

In [45]:
parser_all

[[['n02097658_595.jpg',
   '500',
   '375',
   'silky_terrier',
   '82',
   '378',
   '7',
   '355']],
 [['n02097658_4869.jpg',
   '375',
   '500',
   'silky_terrier',
   '43',
   '321',
   '270',
   '498']],
 [['n02097658_26.jpg',
   '448',
   '400',
   'silky_terrier',
   '41',
   '296',
   '30',
   '398']],
 [['n02097658_422.jpg',
   '500',
   '333',
   'silky_terrier',
   '146',
   '356',
   '10',
   '332']],
 [['n02097658_9222.jpg',
   '333',
   '500',
   'silky_terrier',
   '0',
   '331',
   '12',
   '498']],
 [['n02097658_7970.jpg',
   '375',
   '500',
   'silky_terrier',
   '36',
   '353',
   '11',
   '457']],
 [['n02097658_1326.jpg',
   '375',
   '500',
   'silky_terrier',
   '92',
   '358',
   '105',
   '488']],
 [['n02097658_1469.jpg',
   '375',
   '500',
   'silky_terrier',
   '95',
   '356',
   '37',
   '476']],
 [['n02097658_10567.jpg',
   '500',
   '375',
   'silky_terrier',
   '62',
   '499',
   '11',
   '249']],
 [['n02097658_5223.jpg',
   '375',
   '500',
   'silky_te